In [1]:
# load necessary python modules
import scipy.io as sio
import numpy as np
import rpy2.robjects as rp
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

In [2]:
# define a function to calculate FCD
def FCD(BOLD, window_length, overlap):
    """
    BOLD singal   ....[regions * samples]
    window length ....[samples]
    overlap       ....[samples] 

    output: 
    FCD           ....[number_windows * number_windows]
    """
    import numpy as np
    n_regions = BOLD.shape[0]
    window_steps_size = window_length - overlap;
    n_windows = int(np.round((BOLD.shape[1] - window_length) / window_steps_size + 1))

    #compute FC for each window
    FC_t = np.zeros((n_regions,n_regions,n_windows))
    for i in range(n_windows):
        FC_t[:,:,i] = np.corrcoef(BOLD[:,window_steps_size*i:window_length+window_steps_size*i])
    
    
    # transform FC matrix into vector by just taking the upper triangle
    a,b =np.triu_indices(n_regions,1)
    tFC = FC_t[a,b,:].T
    
    
    # compute FCD, correlate the FCs with each other
    FCD = np.corrcoef(tFC);
    
    return FCD, FC_t

In [3]:
# calculate all empirical FCDs
subj_list = ["CON02T1"]          # specify a list of subject IDs here
path_to_empirical_fmri = "/home/jovyan"  # path to find empirical fmri data
all_FCDs = np.zeros((len(subj_list),64,64)) # initialise array to store FCDs
all_FCDs

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [4]:
# parameters for the FCD
window_length = 30; #[samples]
overlap       = 20; #[samples] 
for i in range(len(subj_list)):    
    #load empirical BOLD data from matlab format
    BOLD = sio.loadmat(path_to_empirical_fmri + '/' + subj_list[i] + '.mat')    
    #compute empirical FCD
    
    all_FCDs[i,:,:], _      = FCD(BOLD,window_length,overlap)
    print(i)

AttributeError: 'dict' object has no attribute 'shape'

In [51]:
BOLD = sio.loadmat("FC.mat")
BOLD

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 4.0.2, 2017-01-23 10:30:38 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'CON02T1_ROIts': array([[ 1223.39299, 12092.66555, 10375.44399, ...,  5130.33761,
         10968.59094, 10996.56872],
        [ 1219.7186 , 12081.63285, 10393.49166, ...,  5039.99379,
         10946.65669, 10947.41631],
        [ 1212.46604, 12083.56704, 10347.83713, ...,  5097.18374,
         10942.06194, 10901.64589],
        ...,
        [ 1223.12041, 12091.89532, 10175.22833, ...,  5107.77232,
         10787.88131, 10862.07812],
        [ 1200.80681, 12025.25664, 10264.57919, ...,  4967.50501,
         10760.85176, 10829.41104],
        [ 1208.18498, 11991.82502, 10269.09345, ...,  5246.64704,
         10925.27825, 10866.34822]]),
 'CON02T1_ROIts_DK68': array([[10917.50963, 15921.57609, 15746.11006, ...,  5130.33761,
         10968.59094, 10996.56872],
        [11011.39579, 15908.80255, 15747.6935 , ...,  5039.99379,
         10946.65669, 10947.41631]

In [63]:
contents = sio.loadmat('FC.mat')
a = contents['ROI_ID_table']
a.shape[0]

112

In [73]:
contents['FC_cc_DK68'].shape[0]

68

In [6]:
# only 4 subjects were simulated with 22 min
# calculate simulated FCDs for each global coupling scaling factor
CSF = np.arange(0.025,0.0401,0.0001)
window_length = 30; #[samples]
overlap       = 20; #[samples]
all_simFCDs = np.zeros((4,151,63,63))
path_to_22min_simulation = ""
for i in range(len(subj_list)):
    for n in range(len(CSF)):
        mat = sio.loadmat(path_to_22min_simulation+subj_list[i]+"/"
                          +subj_list[ind[i]]+"_"+str(np.round(CSF[n],4))+".mat",
                         variable_names="Bold_data")
        BOLD = np.squeeze(mat['Bold_data'])
        BOLD = BOLD[40::4,:].T # cut out first 20s due to gradient, downsample to 0.5 Hz
        
        #compute sim FCD
        all_simFCDs[i,n,:,:], _      = FCD(BOLD,window_length,overlap)

NameError: name 'ind' is not defined

In [ ]:
# compare FCDs using the Kolmogorov-Smirnof distance measure
# import test from R
ks_dist = rp.r('ks.test')
all_KS_dist = np.zeros((4,151))
a,b = np.triu_indices(63,1)
for n in range(4):
    for i in range(151):
        all_KS_dist[n,i] = ks_dist(all_simFCDs[i,a,b],all_FCDs[i,a,b])[0][0]

In [ ]:
# save results 
path_to_save = "" # specify where to save results
sio.savemat(path_to_save+"/FCD_KS_distance.mat",mdict={"all_KS_dist":all_KS_dist})